# Model 1: MultiNomial Naive Bayes

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

## Loading the Dataset

In [29]:
data = pd.read_csv('/Users/tahafaisal/Desktop/ml-news-classification/data/data2cleaned.csv')

data.info()
print()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2086 entries, 0 to 2085
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2086 non-null   int64 
 1   title       2086 non-null   object
 2   link        2086 non-null   object
 3   content     2086 non-null   object
 4   gold_label  2086 non-null   object
dtypes: int64(1), object(4)
memory usage: 81.6+ KB



,id,title,link,content,gold_label
0,0,نوجوت سنگھ سدھو نے 5 برس بعد ’کپل شرما شو‘ چھو...,https://www.express.pk/story/2733251/navjotsin...,نوجوت سنگھ سدھو کی مشہور بھارتی کامیڈی شو ’دی ...,Entertainment
1,1,عدنان صدیقی کی شاہ چارلس سے ملاقات کی تصاویر و...,https://www.express.pk/story/2733247/photos-of...,پاکستان کی شوبز انڈسٹری کے معروف اداکار عدنان ...,Entertainment
2,2,عرفی جاوید کے نئے لباس پر مداح حیران، ویڈیو ان...,https://www.express.pk/story/2733236/fans-are-...,بھارتی ماڈل و اداکار عرفی جاوید کے نئے انداز ن...,Entertainment
3,3,مہوش حیات کو شادی کیلئے کیسا لڑکا چاہیے؟ انٹرو...,https://www.express.pk/story/2733232/what-kind...,پاکستان شوبز کی اداکارہ مہوش حیات کا کہنا ہے ک...,Entertainment
4,4,فیصل قریشی سوشل میڈیا پر تنقید کی زد میں,https://www.express.pk/story/2733220/faisal-qu...,مشہور پاکستانی اداکار فیصل قریشی اور ان کی اہل...,Entertainment


## Preprocessing the Dataset

In [30]:

with open('/Users/tahafaisal/Desktop/ml-news-classification/data/stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

def preprocessor(text):
    
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    return text

data['title'] = data['title'].apply(preprocessor)
data['content'] = data['content'].apply(preprocessor)

data['combined'] = data['title'] + " " + data['content']

data.head()


,id,title,link,content,gold_label,combined
0,0,نوجوت سنگھ سدھو نے 5 برس بعد ’کپل شرما شو‘ چھو...,https://www.express.pk/story/2733251/navjotsin...,نوجوت سنگھ سدھو مشہور بھارتی کامیڈی شو ’دی گری...,Entertainment,نوجوت سنگھ سدھو نے 5 برس بعد ’کپل شرما شو‘ چھو...
1,1,عدنان صدیقی شاہ چارلس سے ملاقات تصاویر وائرل,https://www.express.pk/story/2733247/photos-of...,پاکستان شوبز انڈسٹری معروف اداکار عدنان صدیقی ...,Entertainment,عدنان صدیقی شاہ چارلس سے ملاقات تصاویر وائرل پ...
2,2,عرفی جاوید نئے لباس مداح حیران، ویڈیو انٹرنیٹ ...,https://www.express.pk/story/2733236/fans-are-...,بھارتی ماڈل اداکار عرفی جاوید نئے انداز نے مدا...,Entertainment,عرفی جاوید نئے لباس مداح حیران، ویڈیو انٹرنیٹ ...
3,3,مہوش حیات کو شادی کیسا لڑکا چاہیے؟ انٹرویو وائرل,https://www.express.pk/story/2733232/what-kind...,پاکستان شوبز اداکارہ مہوش حیات کا کہنا وہ اب ش...,Entertainment,مہوش حیات کو شادی کیسا لڑکا چاہیے؟ انٹرویو وائ...
4,4,فیصل قریشی سوشل میڈیا تنقید زد میں,https://www.express.pk/story/2733220/faisal-qu...,مشہور پاکستانی اداکار فیصل قریشی ان اہلیہ ثناء...,Entertainment,فیصل قریشی سوشل میڈیا تنقید زد میں مشہور پاکست...


In [31]:
X = data['content'] 
y = data['gold_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



(1668,) (1668,)
(418,) (418,)


## Implementation of Multinomial Naive Bayes From Scratch

In [32]:
class BagOfWords:
    def __init__(self):
        self.vocabulary = {}  
        self.vocab_size = 0

    def fit(self, corpus):

        unique_words = set()

        for sentence in corpus:
            words = sentence.split() 
            unique_words.update(words)  

        self.vocabulary = {word: idx for idx, word in enumerate(sorted(unique_words))}
        self.vocab_size = len(self.vocabulary)

    def vectorize(self, sentence):

        vector = [0] * self.vocab_size
        
        words = sentence.split()

        for word in words:
            if word in self.vocabulary:
                index = self.vocabulary[word]
                vector[index] += 1

        return vector

In [33]:
model = BagOfWords()
model.fit(X_train)

vector_training = [model.vectorize(doc) for doc in X_train]
vector_test = [model.vectorize(doc) for doc in X_test]


In [34]:
class MultiNaiveBayes:
    def __init__(self):
        self.classprobs = {}  
        self.wordprobs = {}  
        self.vocab_size = 0    
        self.bow = BagOfWords()  # Assume BagOfWords handles vectorization without LabelEncoder                

    def fit(self, X, y):
        self.bow.fit(X)         
        X_vectorized = [self.bow.vectorize(doc) for doc in X] 
        self.vocab_size = self.bow.vocab_size 
        n_docs = len(X)
        
        # Get unique classes as they are
        unique_classes = np.unique(y)  
        class_counts = {c: 0 for c in unique_classes}
        word_counts = {c: np.zeros(self.vocab_size) for c in unique_classes}

        for i in range(n_docs):
            c = y.iloc[i]  # Updated to y.iloc[i] to handle pandas Series correctly
            class_counts[c] += 1
            word_counts[c] += X_vectorized[i]

        for c in unique_classes:
            self.classprobs[c] = class_counts[c] / n_docs
            total_words_in_class = np.sum(word_counts[c])
            self.wordprobs[c] = (word_counts[c] + 1) / (total_words_in_class + self.vocab_size)
    
    def predict(self, X):
        X_vectorized = np.array([self.bow.vectorize(doc) for doc in X])
        predictions = []
        
        for doc_vec in X_vectorized:
            class_scores = {}
            for c in self.classprobs:
                log_prob_c = np.log(self.classprobs[c])
                log_prob_x_given_c = np.sum(doc_vec * np.log(self.wordprobs[c]) + (1 - doc_vec) * np.log(1 - self.wordprobs[c]))
                class_scores[c] = log_prob_c + log_prob_x_given_c
            
            best_class = max(class_scores, key=class_scores.get)
            predictions.append(best_class)
        
        return predictions


In [35]:
model = MultiNaiveBayes()
model.fit(X_train, y_train) 

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
confusionmatrix = confusion_matrix(y_test, predictions)

print("Manually Implemented Naive Bayes:")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print(confusionmatrix)

Manually Implemented Naive Bayes:
Accuracy:  0.8277511961722488
Precision:  0.8605836595532192
Recall:  0.8216063323648817
F1 Score:  0.8361533076233838
[[87  9 10  0  1]
 [ 4 81  7  0  3]
 [ 6  6 78  0  2]
 [ 0  0  5 18  0]
 [ 4  6  9  0 82]]


In [36]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(vector_training, y_train)
mnbprediction = model.predict(vector_test)

accuracy = accuracy_score(y_test, mnbprediction)
precision = precision_score(y_test, mnbprediction, average='macro')
recall = recall_score(y_test, mnbprediction, average='macro')
f1 = f1_score(y_test, mnbprediction, average='macro')
confusionmatrix = confusion_matrix(y_test, mnbprediction)

print("For Sklearn MultiNomial Naive Bayes:")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print(confusionmatrix)
print()

For Sklearn MultiNomial Naive Bayes:
Accuracy:  0.8277511961722488
Precision:  0.8605836595532192
Recall:  0.8216063323648817
F1 Score:  0.8361533076233838
[[87  9 10  0  1]
 [ 4 81  7  0  3]
 [ 6  6 78  0  2]
 [ 0  0  5 18  0]
 [ 4  6  9  0 82]]

